<a href="https://colab.research.google.com/github/tkashdat/Open_Project_Spring_2022/blob/master/tiffany_main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
# ----------------------------------------------------------------------------------------------------------------------
# Name:         tiffany_main_notebook.ipynb
# Purpose:      Open Project Summer - 2022: Predict mortality with medical data
#               CodaLab competition: https://competitions.codalab.org/competitions/27605#learn_the_details
#
# Author(s):    Tiffany Kashima
#
# Created:      6/20/2022
# Updated:      7/21/2022
# Update Comment(s):
#
# TO DO: Finish Pre-Processing, convert to tensors, gridsearch for tuning paramters, model, eval, compare to competition test
#
# FUTURE WORK: Pipeline, productionize
#
# BUGS TO FIX:
#
# ----------------------------------------------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import time
from sklearn.feature_selection import chi2, f_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize

In [3]:
# import libraries

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random as python_random

# set random seeds for reproducibility

np.random.seed(123)
tf.random.set_seed(123)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
# Import training data and feature labels, target

feature_labels = pd.read_csv('/content/drive/MyDrive/Data/mimic_synthetic_feat.csv', header=None).squeeze()
all_train = pd.read_csv('/content/drive/MyDrive/Data/mimic_synthetic_train.csv', delimiter=' ', header=None, names=feature_labels)


In [51]:
all_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79986 entries, 0 to 79998
Columns: 338 entries, INSURANCE to SURVIVED
dtypes: float64(51), int64(280), object(7)
memory usage: 206.9+ MB


In [44]:
all_train

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,GENDER,...,Eyeinfectn,Othereyedx,Otitismedia,Dizziness,Othereardx,Othnervdx,Hrtvalvedx,Carditis,HTN,Htncomplicn
0,45524,108398,2128-05-15_23:42:00,2132-07-23_15:00:00,Private,ENGL,CATHOLIC,DIVORCED,WHITE,F,...,0,0,0,0,0,0.0,0,0,0,0
1,50520,186416,2134-03-17_03:59:00,2113-03-06_12:05:00,Private,ENGL,UNOBTAINABLE,SINGLE,WHITE,M,...,0,0,0,0,0,0.0,0,0,0,1
2,17956,126413,2164-04-05_17:32:00,2180-09-20_16:30:00,Medicaid,SPAN,CATHOLIC,WIDOWED,OTHER,M,...,0,0,0,0,0,0.0,0,0,0,0
3,90258,109355,2102-09-08_00:58:00,2166-06-26_15:30:00,Medicare,ENGL,NOT_SPECIFIED,MARRIED,WHITE,M,...,0,0,0,0,0,0.0,0,0,1,0
4,25159,123784,2163-08-06_12:07:00,2147-01-14_18:40:00,Medicare,ENGL,UNOBTAINABLE,MARRIED,UNKNOWN/NOT_SPECIFIED,F,...,0,0,0,0,0,0.0,0,0,0,0


In [45]:
# Add target variable to dataframe as 'SURVIVED' column

all_train['SURVIVED'] = pd.read_csv('/content/drive/MyDrive/Data/mimic_synthetic_train_labels.csv', header=None).squeeze()


In [46]:
# Drop uninformative features and rows with NA values 

all_train.dropna(inplace=True)
all_train.drop(columns=['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME'], inplace=True)


In [154]:
%%capture
# extract categorical, binary, numerical data features and rearrange

df = all_train.copy()

df['GENDER'] = np.where(df['GENDER'] == 'F', 1, 0)

cats = df.select_dtypes(include=['object'])
cats_labels = list(cats.columns)

binary = pd.DataFrame(df['GENDER'])

nums = df.select_dtypes(exclude=['object'])
nums = nums.astype(int)
nums.drop(columns=['GENDER'], inplace=True)
nums_labels = list(nums.columns)

df = pd.concat([cats, binary, nums], axis=1)

target = df['SURVIVED']

In [155]:
df.head()

,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICU,GENDER,AGE,HR_MIN_DAY1,HR_MAX_DAY1,...,Othereyedx,Otitismedia,Dizziness,Othereardx,Othnervdx,Hrtvalvedx,Carditis,HTN,Htncomplicn,SURVIVED
0,Private,ENGL,CATHOLIC,DIVORCED,WHITE,SICU,1,21,57,108,...,0,0,0,0,0,0,0,0,0,0
1,Private,ENGL,UNOBTAINABLE,SINGLE,WHITE,CCU,0,39,79,121,...,0,0,0,0,0,0,0,0,1,0
2,Medicaid,SPAN,CATHOLIC,WIDOWED,OTHER,MICU,0,40,118,175,...,0,0,0,0,0,0,0,0,0,0
3,Medicare,ENGL,NOT_SPECIFIED,MARRIED,WHITE,CCU,0,75,83,89,...,0,0,0,0,0,0,0,1,0,0
4,Medicare,ENGL,UNOBTAINABLE,MARRIED,UNKNOWN/NOT_SPECIFIED,MICU,1,55,60,105,...,0,0,0,0,0,0,0,0,0,0


In [156]:
# split into train, validation, test sets with 80/20 split, shuffled data

train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])

In [157]:
print(len(train))
print(len(val))
print(len(test))

63988
7999
7999


In [158]:
nums_labels.remove('SURVIVED')

In [163]:
# preprocess data

onehot = OneHotEncoder()
onehot.fit(train[cats_labels])

OneHotEncoder()